In [240]:

MARKET_SHARE = 0.05 # from 
AVERAGE_HOUSEHOLD_SIZE = 2.5 # US average according to ...
# recommended default baseline (actuarial starting point). Tune / replace with real data if available.
BASE_RATE = 1 / 18  # US average annual claim probability per property (sensible starting point)
NUM_TRIALS = 10000  # Monte Carlo draws per county/state

In [241]:
# imports 
import numpy as np
import pandas as pd
import statsmodels.api as sm
import statsmodels.formula.api as smf
from scipy.stats import gamma
from scipy.stats import gamma
from scipy.integrate import quad
from numpy.linalg import cond

In [242]:
# ----------------------------
# Cell A – Data Loading + Preprocessing
# ----------------------------

# Relevant property information
FLORIDA = {
    "name": "Florida",
    "average_property_cost": 325000, 
}
WASHINGTON = {
    "name": "Washington",
    "average_property_cost": 610000, 
}

# Load FEMA CSVs
FLORIDA['df'] = pd.read_csv('fl_county.csv')
WASHINGTON['df'] = pd.read_csv('wa_county.csv')

# ----------------------------
# CLEAN + PREPARE FEMA DATA
# ----------------------------
def clean_fema_df(state_dict):
    df = state_dict["df"].copy()

    # Keep only the necessary columns
    needed = ["COUNTY", "POPULATION", "BUILDVALUE", "EAL_VALT", "RISK_VALUE", "RISK_SCORE"]
    df = df[needed]

    # Rename for clarity
    df = df.rename(columns={
        "BUILDVALUE": "exposure",  # total building value
        "EAL_VALT": "EAL",         # total expected annual loss
        "POPULATION": "population"
    })

    # Convert numeric
    df["exposure"] = pd.to_numeric(df["exposure"], errors="coerce")
    df["EAL"] = pd.to_numeric(df["EAL"], errors="coerce")
    df["population"] = pd.to_numeric(df["population"], errors="coerce")
    df["RISK_VALUE"] = pd.to_numeric(df["RISK_VALUE"], errors="coerce")
    df["RISK_SCORE"] = pd.to_numeric(df["RISK_SCORE"], errors="coerce")

    # Drop rows with missing exposure or EAL
    df = df.dropna(subset=["exposure", "EAL"])

    state_dict["df_clean"] = df
    return state_dict

FLORIDA = clean_fema_df(FLORIDA)
WASHINGTON = clean_fema_df(WASHINGTON)

# ----------------------------
# Preprocessing for GLM / Monte Carlo
# ----------------------------

def preprocess_location_df(location):
    """
    Produces county-level variables for frequency × severity modeling:
    - insured_properties: population / average household size
    - lambda_base: expected claims per county based on empirical claim rate (~1/18)
    - severity_mean: EAL per expected claim
    - risk_scaled: normalized FEMA RISK_VALUE
    """
    df = location["df_clean"].copy()

    # Estimate # of properties
    df["properties"] = df["population"] / AVERAGE_HOUSEHOLD_SIZE

    # Assume ~1/18 of properties claim per year
    base_claim_rate = 1/18
    df["insured_properties"] = df["properties"] * base_claim_rate

    # baseline lambda = insured properties * empirical base rate
    df["lambda_base"] = df["insured_properties"] * BASE_RATE


    # severity per claim
    df["severity_mean"] = df["EAL"] / df["lambda_base"]

    # normalized risk factor
    df["risk_scaled"] = df["RISK_VALUE"] / df["RISK_VALUE"].max()

    location["df_glm"] = df
    return df

FLORIDA["df_glm"] = preprocess_location_df(FLORIDA)
WASHINGTON["df_glm"] = preprocess_location_df(WASHINGTON)

# Quick sanity check
print("Sample counties for Florida:")
print(FLORIDA["df_glm"][["COUNTY", "insured_properties", "lambda_base", "severity_mean"]].head())


Sample counties for Florida:
     COUNTY  insured_properties  lambda_base  severity_mean
0   Alachua         6177.422222   343.190123  196545.729472
1     Baker          618.933333    34.385185  137107.148543
2       Bay         3885.977778   215.887654  712090.788201
3  Bradford          627.400000    34.855556  239810.046710
4   Brevard        13468.155556   748.230864  553818.713457


In [243]:
#Cell B

def generate_county_level_parameters(location, perturb_std_lambda=0.05, perturb_std_severity=0.1, random_state=None):
    """
    Produce county-level stochastic frequency (lambda) and severity.
    
    Args:
        location: dict containing df_clean / df_glm
        perturb_std_lambda: relative std dev for lambda perturbation (5% default)
        perturb_std_severity: relative std dev for severity perturbation (10% default)
        random_state: seed for reproducibility
    """
    rng = np.random.default_rng(random_state)
    df = location["df_glm"].copy()
    
    # --- Perturb lambda around base ---
    df["lambda"] = df["lambda_base"] * (1 + rng.normal(0, perturb_std_lambda, size=len(df)))
    df["lambda"] = df["lambda"].clip(lower=0.01)  # prevent negative lambda

    # --- Perturb severity around mean ---
    df["severity_mean"] = df["severity_mean"] * (1 + rng.normal(0, perturb_std_severity, size=len(df)))
    df["severity_mean"] = df["severity_mean"].clip(lower=1000)  # prevent absurdly low severity

    
    location["df_sim"] = df
    return location

# Apply to both states
FLORIDA = generate_county_level_parameters(FLORIDA, random_state=42)
WASHINGTON = generate_county_level_parameters(WASHINGTON, random_state=42)

# Quick check
FLORIDA["df_sim"][["COUNTY", "insured_properties", "lambda", "severity_mean"]].head()


,COUNTY,insured_properties,lambda,severity_mean
0,Alachua,6177.422222,348.418918,187458.402435
1,Baker,618.933333,32.597183,148870.611203
2,Bay,3885.977778,223.988312,698468.183451
3,Bradford,627.400000,36.494751,209217.807032
4,Brevard,13468.155556,675.239627,491055.146773


In [244]:
# --- Cell D: Monte Carlo Simulation with Quadrature for Severity ---

from scipy.stats import gamma
import numpy as np

def expected_payout_quadrature(deductible, coinsurance, shape, scale, n=200):
    """
    Computes E[(X - deductible)^+] * coinsurance for Gamma(shape, scale)
    using n-point Gauss–Legendre quadrature.
    """
    # Upper limit chosen as 99.99th percentile of severity distribution
    upper = gamma.ppf(0.9999, a=shape, scale=scale)
    
    # Gauss–Legendre nodes & weights on [-1,1]
    xs, ws = np.polynomial.legendre.leggauss(n)
    
    # Transform to [deductible, upper]
    t = 0.5 * (xs + 1) * (upper - deductible) + deductible
    w = 0.5 * (upper - deductible) * ws

    # Integrand: (x - d) * f_X(x)
    integrand = (t - deductible) * gamma.pdf(t, a=shape, scale=scale)
    
    # Multiply by coinsurance factor
    return coinsurance * np.sum(w * integrand)


def apply_deductible_coinsurance_vectorized(county_df, deductible=10000, coinsurance=0.8,
                                            n_sim=NUM_TRIALS, random_state=42):
    """
    Monte Carlo simulation of state losses:
        - Poisson frequency simulation
        - Expected severity payout computed via Gauss–Legendre quadrature
    """
    rng = np.random.default_rng(random_state)
    state_losses = np.zeros(n_sim)

    for _, row in county_df.iterrows():
        
        # --- 1. Simulate claim frequency for this county ---
        n_claims = rng.poisson(lam=row['lambda'], size=n_sim)

        # --- 2. Quadrature: expected payout per claim ---
        severity_shape = 4  # can be tuned
        severity_scale = row['severity_mean'] / severity_shape

        E_payout = expected_payout_quadrature(
            deductible=deductible,
            coinsurance=coinsurance,
            shape=severity_shape,
            scale=severity_scale
        )

        # --- 3. Total losses per simulation draw ---
        county_total = n_claims * E_payout

        # accumulate into state-level losses
        state_losses += county_total

    return state_losses


# Quick sense check with first 5 counties
FL_sim_losses = apply_deductible_coinsurance_vectorized(FLORIDA['df_sim'].head(5), n_sim=NUM_TRIALS)
WA_sim_losses = apply_deductible_coinsurance_vectorized(WASHINGTON['df_sim'].head(5), n_sim=NUM_TRIALS)

print(f"Florida sample losses (first 5 counties, {NUM_TRIALS} draws):")
print(pd.Series(FL_sim_losses).describe())

print(f"\nWashington sample losses (first 5 counties, {NUM_TRIALS} draws):")
print(pd.Series(WA_sim_losses).describe())


Florida sample losses (first 5 counties, 10000 draws):
count    1.000000e+04
mean     4.419479e+08
std      1.335672e+07
min      3.868385e+08
25%      4.328650e+08
50%      4.417228e+08
75%      4.510809e+08
max      4.886637e+08
dtype: float64

Washington sample losses (first 5 counties, 10000 draws):
count    1.000000e+04
mean     6.513420e+07
std      3.212604e+06
min      5.341716e+07
25%      6.300491e+07
50%      6.511345e+07
75%      6.724170e+07
max      7.852265e+07
dtype: float64


In [245]:
# --- Cell E: Metrics ---
def compute_metrics(state_losses, loading=1.2, n_bootstrap=NUM_TRIALS, random_state=42):
    """
    Compute premium, profit samples, VaR, TVaR, and bootstrap CI.
    Inputs:
        state_losses: array of total state-level losses (after deductible & coinsurance)
        loading: premium multiplier
        n_bootstrap: number of bootstrap resamples for mean profit CI
    Returns:
        metrics: dict with expected loss, premium, VaR, TVaR, profit samples, bootstrap CI
    """
    rng = np.random.default_rng(random_state)
    state_losses = np.array(state_losses)
    
    # expected loss and premium
    expected_loss = state_losses.mean()
    premium = expected_loss * loading
    
    # profit samples
    profit_samples = premium - state_losses
    
    # risk metrics
    VaR_95 = np.percentile(profit_samples, 5)
    TVaR_95 = profit_samples[profit_samples <= VaR_95].mean()
    
    # bootstrap CI for mean profit
    bootstrap_means = np.array([rng.choice(profit_samples, size=len(profit_samples), replace=True).mean() for _ in range(n_bootstrap)])
    ci_lower, ci_upper = np.percentile(bootstrap_means, [2.5, 97.5])
    
    metrics = {
        'expected_loss': expected_loss,
        'premium': premium,
        'VaR_95': VaR_95,
        'TVaR_95': TVaR_95,
        'profit_mean': profit_samples.mean(),
        'profit_std': profit_samples.std(),
        'profit_bootstrap_CI': (ci_lower, ci_upper)
    }
    
    return metrics

# --- Quick sense check ---
FL_metrics = compute_metrics(FL_sim_losses)
WA_metrics = compute_metrics(WA_sim_losses)

print("Florida metrics:")
for k, v in FL_metrics.items():
    print(k, v)

print("\nWashington metrics:")
for k, v in WA_metrics.items():
    print(k, v)


Florida metrics:
expected_loss 441947927.29505956
premium 530337512.7540715
VaR_95 66432449.776738405
TVaR_95 60847769.847131684
profit_mean 88389585.45901182
profit_std 13356048.414985042
profit_bootstrap_CI (np.float64(88125034.92020176), np.float64(88652989.35282354))

Washington metrics:
expected_loss 65134201.33090066
premium 78161041.5970808
VaR_95 7751648.710351333
TVaR_95 6288408.535639963
profit_mean 13026840.266180133
profit_std 3212442.9793348312
profit_bootstrap_CI (np.float64(12962766.545188608), np.float64(13090408.111911109))


In [246]:
# ----------------------------
# Cell F – Diagnostics
# ----------------------------

epsilon = 1e-5  # for finite-difference gradient

# --- Frequency GLM (Poisson) ---
df = FLORIDA['df_glm'].copy()
y_freq = df['insured_properties']
X_freq = sm.add_constant(df[['risk_scaled']])  # example covariate

freq_model = sm.GLM(y_freq, X_freq, family=sm.families.Poisson())
freq_results = freq_model.fit()
print("Frequency GLM summary:")
print(freq_results.summary())

# Finite-difference gradient check
beta = freq_results.params.values
grad_approx = np.zeros_like(beta)
for i in range(len(beta)):
    beta_plus = beta.copy(); beta_plus[i] += epsilon
    beta_minus = beta.copy(); beta_minus[i] -= epsilon
    mu_plus = np.exp(X_freq @ beta_plus)
    mu_minus = np.exp(X_freq @ beta_minus)
    grad_approx[i] = (mu_plus.sum() - mu_minus.sum()) / (2*epsilon)

# Analytical gradient (score function)
mu = freq_results.fittedvalues
grad_analytical = X_freq.values.T @ (y_freq.values - mu)

print("\nFrequency GLM gradients:")
print("Approximate (finite-diff):", grad_approx)
print("Analytical:", grad_analytical)

# Condition number
X_freq_cond = cond(X_freq.values)
print("Frequency GLM design matrix condition number:", X_freq_cond)

# --- Severity GLM (Gamma, log link) ---
y_sev = df['severity_mean']
X_sev = sm.add_constant(df[['risk_scaled']])  # same covariate

sev_model = sm.GLM(y_sev, X_sev, family=sm.families.Gamma(sm.families.links.log()))
sev_results = sev_model.fit()
print("\nSeverity GLM summary:")
print(sev_results.summary())

# Finite-difference gradient check
beta_sev = sev_results.params.values
grad_approx_sev = np.zeros_like(beta_sev)
for i in range(len(beta_sev)):
    beta_plus = beta_sev.copy(); beta_plus[i] += epsilon
    beta_minus = beta_sev.copy(); beta_minus[i] -= epsilon
    mu_plus = np.exp(X_sev @ beta_plus)
    mu_minus = np.exp(X_sev @ beta_minus)
    grad_approx_sev[i] = (mu_plus.sum() - mu_minus.sum()) / (2*epsilon)

# Analytical gradient (simplified)
mu_sev = sev_results.fittedvalues
grad_analytical_sev = X_sev.values.T @ ((y_sev.values - mu_sev) / (mu_sev ** 2))

print("\nSeverity GLM gradients:")
print("Approximate (finite-diff):", grad_approx_sev)
print("Analytical:", grad_analytical_sev)

# Condition number
X_sev_cond = cond(X_sev.values)
print("Severity GLM design matrix condition number:", X_sev_cond)


Frequency GLM summary:
                 Generalized Linear Model Regression Results                  
Dep. Variable:     insured_properties   No. Observations:                   67
Model:                            GLM   Df Residuals:                       65
Model Family:                 Poisson   Df Model:                            1
Link Function:                    Log   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:            -1.3547e+05
Date:                Fri, 12 Dec 2025   Deviance:                   2.7028e+05
Time:                        00:08:17   Pearson chi2:                 3.09e+05
No. Iterations:                     6   Pseudo R-squ. (CS):              1.000
Covariance Type:            nonrobust                                         
                  coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------
const           8.1538     

/Users/junomarquesoda/anaconda3/lib/python3.11/site-packages/statsmodels/genmod/families/links.py:13: FutureWarning: The log link alias is deprecated. Use Log instead. The log link alias will be removed after the 0.15.0 release.
  warnings.warn(
